In [1]:
import numpy as np
import scipy
import sklearn
import _pickle
import matplotlib.pyplot as plt
from IPython.display import display, Image
from scipy import ndimage

import tensorflow as tf

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
i=1
cifar_data = np.empty(shape=(10000,3072),dtype='uint8')
labels = np.empty(shape=(10000,),dtype='int32')
while i<=1:
    with open(str.format("cifar-10-batches-py\data_batch_{0}",i), 'rb') as cifar_batch:
        batch_dict = _pickle.load(cifar_batch,encoding='bytes')
        cifar_data[(i-1)*10000:i*10000] = batch_dict[b'data'].astype('float32')
        labels[(i-1)*10000:i*10000] = batch_dict[b'labels']
    i += 1
    
cifar_data = (cifar_data.astype('float32'))/255

In [ ]:
## placeholder for input. In this case CIFAR-10 dataset which is N x 3072
x = tf.placeholder(tf.float32, [None, 3072])

## Initializing tensforflow variables W and b
W = tf.Variable(tf.zeros([3072,10]))
b = tf.Variable(tf.zeros(10))

## Implementing softmax
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.int32, [None,])
y_one_hot = tf.one_hot(indices=y_,depth=10,on_value=1,off_value=0,axis=-1)

#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_one_hot, logits=y))
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=y_))

## Minimizing Cross entropy
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(cross_entropy)

## Prediction and Accuracy
z = tf.to_int64(y_)
correct_prediction = tf.equal(tf.argmax(y, 1), z)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
## Interactive Session
sess = tf.InteractiveSession()

## Initialize the variables
tf.global_variables_initializer().run()

print(sess.run(accuracy, feed_dict={x: cifar_data, y_:labels}))